In [ ]:
# reference: https://www.kaggle.com/code/ankasor/improved-lung-segmentation-using-watershed/notebook

In [ ]:
%matplotlib inline

import numpy as np
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
import glob

from skimage import measure, segmentation
from tqdm import tqdm

from utils import (
    plot_3d, load_scan,
    optimized_separate_lungs
)

# Some constants 
INPUT_FOLDER = r'D:/masters_datasets/LUNA16-original/LUNA16_Images/*.mhd'
exams = glob.glob(INPUT_FOLDER)
print(len(exams))
exams.sort()

### Loading the files

In [ ]:
first_exam_pixels = load_scan(exams[7])
print(first_exam_pixels.shape)
print ("Original Slice")
slice = 100
plt.imshow(first_exam_pixels[slice], cmap=plt.cm.gray)
plt.show()

In [ ]:
print(first_exam_pixels.shape)

### Generate markers

In [ ]:
def generate_markers(image):
    #Creation of the internal Marker
    marker_internal = image < -400
    marker_internal = segmentation.clear_border(marker_internal)
    marker_internal_labels = measure.label(marker_internal)
    areas = [r.area for r in measure.regionprops(marker_internal_labels)]
    areas.sort()
    if len(areas) > 2:
        for region in measure.regionprops(marker_internal_labels):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                       marker_internal_labels[coordinates[0], coordinates[1]] = 0
    marker_internal = marker_internal_labels > 0
    #Creation of the external Marker
    external_a = ndimage.binary_dilation(marker_internal, iterations=10)
    external_b = ndimage.binary_dilation(marker_internal, iterations=55)
    marker_external = external_b ^ external_a
    #Creation of the Watershed Marker matrix
    marker_watershed = np.zeros((512, 512), dtype=int)
    marker_watershed += marker_internal * 255
    marker_watershed += marker_external * 128
    
    return marker_internal, marker_external, marker_watershed

#Show some example markers from the middle        
# test_patient_internal, test_patient_external, test_patient_watershed = generate_markers(first_exam_pixels[slice])
# print ("Internal Marker")
# plt.imshow(test_patient_internal, cmap='gray')
# plt.show()
# print ("External Marker")
# plt.imshow(test_patient_external, cmap='gray')
# plt.show()
# print ("Watershed Marker")
# plt.imshow(test_patient_watershed, cmap='gray')
# plt.show()

### Separate lungs

In [ ]:
def seperate_lungs(image):
    #Creation of the markers as shown above:
    marker_internal, marker_external, marker_watershed = generate_markers(image)
    
    #Creation of the Sobel-Gradient
    sobel_filtered_dx = ndimage.sobel(image, 1)
    sobel_filtered_dy = ndimage.sobel(image, 0)
    sobel_gradient = np.hypot(sobel_filtered_dx, sobel_filtered_dy)
    sobel_gradient *= 255.0 / np.max(sobel_gradient)
    
    #Watershed algorithm
    watershed = segmentation.watershed(sobel_gradient, marker_watershed)
    
    #Reducing the image created by the Watershed algorithm to its outline
    outline = ndimage.morphological_gradient(watershed, size=(3,3))
    outline = outline.astype(bool)
    
    #Performing Black-Tophat Morphology for reinclusion
    #Creation of the disk-kernel and increasing its size a bit
    blackhat_struct = [[0, 0, 1, 1, 1, 0, 0],
                       [0, 1, 1, 1, 1, 1, 0],
                       [1, 1, 1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1, 1, 1],
                       [0, 1, 1, 1, 1, 1, 0],
                       [0, 0, 1, 1, 1, 0, 0]]
    blackhat_struct = ndimage.iterate_structure(blackhat_struct, 8)
    #Perform the Black-Hat
    outline += ndimage.black_tophat(outline, structure=blackhat_struct)
    
    #Use the internal marker and the Outline that was just created to generate the lungfilter
    lungfilter = np.bitwise_or(marker_internal, outline)
    #Close holes in the lungfilter
    #fill_holes is not used here, since in some slices the heart would be reincluded by accident
    lungfilter = ndimage.morphology.binary_closing(lungfilter, structure=np.ones((5,5)), iterations=3)
    
    #Apply the lungfilter (note the filtered areas being assigned -2000 HU)
    segmented = np.where(lungfilter == 1, image, -2000*np.ones((512, 512)))
    
    return segmented, lungfilter, outline, watershed, sobel_gradient, marker_internal, marker_external, marker_watershed

#Some Testcode:
# (test_segmented, test_lungfilter, test_outline, 
#  test_watershed, test_sobel_gradient, test_marker_internal, 
#  test_marker_external, test_marker_watershed) = seperate_lungs(
#      first_exam_pixels[slice]
# )

# print ("Sobel Gradient")
# plt.imshow(test_sobel_gradient, cmap='gray')
# plt.show()
# print ("Watershed Image")
# plt.imshow(test_watershed, cmap='gray')
# plt.show()
# print ("Outline after reinclusion")
# plt.imshow(test_outline, cmap='gray')
# plt.show()
# print ("Lungfilter after closing")
# plt.imshow(test_lungfilter, cmap='gray')
# plt.show()
# print ("Segmented Lung")
# plt.imshow(test_segmented, cmap='gray')
# plt.show()

### multiple layers

In [ ]:
def segmented_entire_image(image):
    for i, axial_slice in tqdm(enumerate(image)):
        image[i] = optimized_separate_lungs(axial_slice)

    return image

In [ ]:
# segmented_lungs = segmented_entire_image(first_exam_pixels)
# print(segmented_lungs.shape)

In [ ]:
# plot_3d(segmented_lungs, 0)

### Batch visualization

In [ ]:
for i in tqdm(range(582, 887)):
    current_exam = load_scan(exams[i])
    # im = sitk.GetImageFromArray(current_exam)
    # sitk.WriteImage(im, 'generated/exam.nrrd', True)

    segmented_lungs = segmented_entire_image(current_exam)
    # ma = sitk.GetImageFromArray(segmented_lungs)
    # sitk.WriteImage(ma, 'generated/mask.nrrd', True)
    output = np.unique(segmented_lungs, return_counts=True)
    name = str(i) + '-' + str(round(output[1][1]/output[1][0], 5))
    plot_3d(segmented_lungs, 0, name='visu-watershed/'+name)